In [2]:
from bs4 import BeautifulSoup
import requests
import datetime
import calendar
import pandas as pd

In [3]:
# URL to download is a jsp page with params
# example below
# https://www.nseindia.com/products/dynaContent/equities/indices/historical_pepb.jsp?indexName=NIFTY%2050&fromDate=01-01-2018&toDate=12-06-2018&yield1=undefined&yield2=undefined&yield3=undefined&yield4=all
# The data returned will only return 100 rows at a time,
# so we can download with a rolling window smaller than that

In [4]:
# Just a test run
#r = requests.get('https://www.nseindia.com/products/dynaContent/equities/indices/total_returnindices.jsp?indexType=NIFTY%2050&fromDate=01-01-2015&toDate=31-12-2015')
r = requests.get('https://www.nseindia.com/products/dynaContent/equities/indices/historical_pepb.jsp?indexName=NIFTY%2050&fromDate=01-01-2018&toDate=12-06-2018&yield1=undefined&yield2=undefined&yield3=undefined&yield4=all')
print(r)

<Response [200]>


In [5]:
NIFTY_PE_URL="https://www.nseindia.com/products/dynaContent/equities/indices/historical_pepb.jsp?indexName={index}&fromDate={fromDate}&toDate={toDate}&yield1=undefined&yield2=undefined&yield3=undefined&yield4=all"
def create_nifty_yield_url(index, from_date, to_date):
    #print(from_date)
    #print(to_date)
    u = NIFTY_PE_URL.format(index=index, 
                                fromDate=from_date.strftime("%d-%m-%Y"), 
                                toDate=to_date.strftime("%d-%m-%Y"))
    return u

print(create_nifty_yield_url("NIFTY 50", datetime.date(2000, 1, 1), datetime.date(2000, 1, 31)))

https://www.nseindia.com/products/dynaContent/equities/indices/historical_pepb.jsp?indexName=NIFTY 50&fromDate=01-01-2000&toDate=31-01-2000&yield1=undefined&yield2=undefined&yield3=undefined&yield4=all


In [6]:
r = requests.get(create_nifty_yield_url("NIFTY 50", datetime.date(2000, 1, 1), datetime.date(2000, 1, 31)))
sampletext = r.text
print(sampletext)

 

<!-- VAPT Azhar-->


<!-- VAPT Azhar-->






				<TABLE width="80%"><tbody>
     	    	<TR>
					<Th colspan="4" >Historical NIFTY 50  P/E, P/B & Div. Yield values</th>
				</tr>
				<tr><th colspan="4">
					For the period 01-01-2000 to 31-01-2000
					</th>
				</tr>
     	    	
     	    
     	    	<!-- Table heading -->
     	    	<tr>
     	    	
     	    	
     	    	 <th>Date </th>
     	    	
     	    	 <th>P/E </th>
     	    	
     	    	 <th>P/B </th>
     	    	
     	    	 <th>Div Yield </th>
     	    	
     	    	
     	    	</tr>
     	            
     	    	   <tr>
     	    	   
     	    	  	  <td class="date">03-Jan-2000</td>
     	    	  	  
     	    	  	  <td class="number">       25.91</td>
     	    	  	  
     	    	  	  <td class="number">        4.63</td>
     	    	  	  
     	    	  	  <td class="number">        0.95</td>
     	    	  	  
     	    	   </tr>
     	    	   
     	    	   
     	    	   <tr>
     	    	   
     	    	  	  <td class="

In [7]:
def gen_input_range(sYear, sMonth, eYear, eMonth):
    ir = []
    for y in range(sYear, eYear+1):
        sm = 1
        em = 12
        if sYear == y:
            sm = sMonth
        if eYear == y:
            em = eMonth
        for m in range(sm, em+1):
            mr = calendar.monthrange(y, m)
            #print("{year} {month} {mr}".format(year=y, month=m, mr=mr))
            ir.append((datetime.date(y, m, 1), datetime.date(y, m, mr[1])))
    #print(ir)
    return ir
input_range = gen_input_range(2000, 2, 2001, 4)
for i in input_range:
    print(create_nifty_yield_url("NIFTY 50", i[0], i[1]))

https://www.nseindia.com/products/dynaContent/equities/indices/historical_pepb.jsp?indexName=NIFTY 50&fromDate=01-02-2000&toDate=29-02-2000&yield1=undefined&yield2=undefined&yield3=undefined&yield4=all
https://www.nseindia.com/products/dynaContent/equities/indices/historical_pepb.jsp?indexName=NIFTY 50&fromDate=01-03-2000&toDate=31-03-2000&yield1=undefined&yield2=undefined&yield3=undefined&yield4=all
https://www.nseindia.com/products/dynaContent/equities/indices/historical_pepb.jsp?indexName=NIFTY 50&fromDate=01-04-2000&toDate=30-04-2000&yield1=undefined&yield2=undefined&yield3=undefined&yield4=all
https://www.nseindia.com/products/dynaContent/equities/indices/historical_pepb.jsp?indexName=NIFTY 50&fromDate=01-05-2000&toDate=31-05-2000&yield1=undefined&yield2=undefined&yield3=undefined&yield4=all
https://www.nseindia.com/products/dynaContent/equities/indices/historical_pepb.jsp?indexName=NIFTY 50&fromDate=01-06-2000&toDate=30-06-2000&yield1=undefined&yield2=undefined&yield3=undefined&y

In [8]:
ssoup = BeautifulSoup(sampletext, "html5lib")

In [10]:
items = []
print(items)
for row in ssoup.find_all('tr'):
    item = []
    columns = row.find_all('td')
    has_content = False
    for column in columns:
        if not 'csv' in column.get_text():
            item.append(column.get_text())
        else:
            break
        #print(column.get_text())
        has_content = True
    if has_content:
        items.append(item)
    
#print(items)

new_table = pd.DataFrame(items, columns=['Date', 'PE', 'PB', 'DivYield'])
new_table.head()

[]


,Date,PE,PB,DivYield
0,03-Jan-2000,25.91,4.63,0.95
1,04-Jan-2000,26.67,4.76,0.92
2,05-Jan-2000,25.97,4.64,0.95
3,06-Jan-2000,26.32,4.70,0.94
4,07-Jan-2000,26.25,4.69,0.94


In [17]:
def download_and_parse_yield(url):
    r = requests.get(url)
    ssoup = BeautifulSoup(r.text, "html5lib")
    items = []
    for row in ssoup.find_all('tr'):
        item = []
        columns = row.find_all('td')
        has_content = False
        for column in columns:
            if not 'csv' in column.get_text():
                item.append(column.get_text().strip())
            else:
                break
            #print(column.get_text())
            has_content = True
        if has_content:
            items.append(item)

    #print(items)

    new_table = pd.DataFrame(items, columns=['Date', 'PE', 'PB', 'DivYield'])
    new_table.head()
    return new_table

In [19]:
def download_all_nifty50_yield(start_year, start_month, end_year, end_month):
    months = {}
    input_range = gen_input_range(start_year, start_month, end_year, end_month)
    for i in input_range:
        months[i] = create_nifty_yield_url("NIFTY 50", i[0], i[1])
    #print (months)
    dflist = []
    for m in months.keys():
        df = download_and_parse_yield(months[m])
        dflist.append(df)
        print(m)
    print ('Done')
    alldf = pd.concat(dflist)
    alldf.head()
    print(alldf.tail())
    alldf.to_csv('./data/nifty50-yield-{}-{}-{}-{}.csv'.format(start_year, start_month, end_year, end_month), index=False)
    
download_all_nifty50_yield(2000, 1, 2018, 5)

(datetime.date(2000, 1, 1), datetime.date(2000, 1, 31))
(datetime.date(2000, 2, 1), datetime.date(2000, 2, 29))
(datetime.date(2000, 3, 1), datetime.date(2000, 3, 31))
(datetime.date(2000, 4, 1), datetime.date(2000, 4, 30))
(datetime.date(2000, 5, 1), datetime.date(2000, 5, 31))
(datetime.date(2000, 6, 1), datetime.date(2000, 6, 30))
(datetime.date(2000, 7, 1), datetime.date(2000, 7, 31))
(datetime.date(2000, 8, 1), datetime.date(2000, 8, 31))
(datetime.date(2000, 9, 1), datetime.date(2000, 9, 30))
(datetime.date(2000, 10, 1), datetime.date(2000, 10, 31))
(datetime.date(2000, 11, 1), datetime.date(2000, 11, 30))
(datetime.date(2000, 12, 1), datetime.date(2000, 12, 31))
(datetime.date(2001, 1, 1), datetime.date(2001, 1, 31))
(datetime.date(2001, 2, 1), datetime.date(2001, 2, 28))
(datetime.date(2001, 3, 1), datetime.date(2001, 3, 31))
(datetime.date(2001, 4, 1), datetime.date(2001, 4, 30))
(datetime.date(2001, 5, 1), datetime.date(2001, 5, 31))
(datetime.date(2001, 6, 1), datetime.date(

(datetime.date(2012, 3, 1), datetime.date(2012, 3, 31))
(datetime.date(2012, 4, 1), datetime.date(2012, 4, 30))
(datetime.date(2012, 5, 1), datetime.date(2012, 5, 31))
(datetime.date(2012, 6, 1), datetime.date(2012, 6, 30))
(datetime.date(2012, 7, 1), datetime.date(2012, 7, 31))
(datetime.date(2012, 8, 1), datetime.date(2012, 8, 31))
(datetime.date(2012, 9, 1), datetime.date(2012, 9, 30))
(datetime.date(2012, 10, 1), datetime.date(2012, 10, 31))
(datetime.date(2012, 11, 1), datetime.date(2012, 11, 30))
(datetime.date(2012, 12, 1), datetime.date(2012, 12, 31))
(datetime.date(2013, 1, 1), datetime.date(2013, 1, 31))
(datetime.date(2013, 2, 1), datetime.date(2013, 2, 28))
(datetime.date(2013, 3, 1), datetime.date(2013, 3, 31))
(datetime.date(2013, 4, 1), datetime.date(2013, 4, 30))
(datetime.date(2013, 5, 1), datetime.date(2013, 5, 31))
(datetime.date(2013, 6, 1), datetime.date(2013, 6, 30))
(datetime.date(2013, 7, 1), datetime.date(2013, 7, 31))
(datetime.date(2013, 8, 1), datetime.date(